In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split


spark=(
    SparkSession
    .builder
    .appName("StructuredWordCount")
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.1")
    .config("spark.streaming.stopGracefullyOnShutdown", "true")
    .config("spark.sql.shuffle.partitions", "3")
    .getOrCreate()
)

events=(
    spark
    .readStream
    .format("kafka")
    .option("kafka.bootstrap.servers","kafka:9092")
    .option("subscribe", "test-topic")
    .option("startingOffsets","earliest")
    .load()
)
events.printSchema()

words = events.select(
   explode(
       split(events.value, " ")
   ).alias("word")
)

wordCounts = words.groupBy("word").count()

query=(
    wordCounts
    .writeStream
    .option("checkpointLocation", "checkpoint")
    .outputMode("complete")
    .format("console")
    .start()
)
query.awaitTermination()

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-8e1a4e02-82de-409f-9bd1-02d9fc64d5bc;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.1 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.3 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
downloading https://repo1.maven.org/maven2/org/apache/spark/spark-sql-kafka-

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



26/02/03 07:52:45 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
26/02/03 07:52:45 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.
                                                                                

-------------------------------------------
Batch: 0
-------------------------------------------
+----------+-----+
|      word|count|
+----------+-----+
|안녕하세요|    1|
+----------+-----+

-------------------------------------------
Batch: 1
-------------------------------------------
+----------+-----+
|      word|count|
+----------+-----+
|안녕하세요|    1|
|      ��Hi|    1|
+----------+-----+

-------------------------------------------
Batch: 2
-------------------------------------------
+----------+-----+
|      word|count|
+----------+-----+
|안녕하세요|    1|
|      ��Hi|    1|
|      안녕|    1|
+----------+-----+

-------------------------------------------
Batch: 3
-------------------------------------------
+----------+-----+
|      word|count|
+----------+-----+
|    친구야|    1|
|  신기하네|    1|
|안녕하세요|    1|
|      ��Hi|    1|
|      안녕|    1|
+----------+-----+

-------------------------------------------
Batch: 4
-------------------------------------------
+----------+-----+
|      wor

-------------------------------------------
Batch: 5
-------------------------------------------
+----------+-----+
|      word|count|
+----------+-----+
|    친구야|    1|
|      바이|    1|
|  신기하네|    1|
|안녕하세요|    1|
|   kafka야|    1|
|      ��Hi|    1|
|      안녕|    2|
|    안안녕|    1|
+----------+-----+



ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
RuntimeError: reentrant call inside <_io.BufferedReader name=59>

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.8/dist-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.8/socket.py

Py4JError: An error occurred while calling o58.awaitTermination